# Reading events from RGTDB

Query to get JSON:

https://rgtdb.com/events/json?search=&offset=0&limit=100

This reads upcoming events from rgtdb.com and converts them into iCAL format and writes to a file for manual import. Then this saves or updates the events in a Google calendar.

## Get data and cache response

In [1]:
import pandas as pd 

import datetime
from datetime import datetime as dt

import io
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("Debug level logging turned on")

import requests
from cachecontrol import CacheControl
from cachecontrol.caches import FileCache
from cachecontrol.heuristics import ExpiresAfter

sess = requests.session()
cached_sess = CacheControl(sess, cache = FileCache('.web_cache'), heuristic=ExpiresAfter(hours=1))

try:
    response = cached_sess.get('https://rgtdb.com/events/json?search=&offset=0&limit=200') # Get 200 events. Should be about a week's worth of events
    response.raise_for_status()

except HTTPError as http_err:
    print(f'HTTP error occurred: {http_err}')
except Exception as err:
    print(f'Other error occurred: {err}')

logger.setLevel(logging.ERROR)

print(dt.now())

DEBUG:root:Debug level logging turned on
DEBUG:cachecontrol.controller:Looking up "https://rgtdb.com/events/json?search=&offset=0&limit=200" in the cache
DEBUG:cachecontrol.controller:Current age based on date: 26637
DEBUG:cachecontrol.controller:Freshness lifetime from expires: 3600
DEBUG:cachecontrol.controller:The cached response is "stale" with no etag, purging
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): rgtdb.com:443
DEBUG:urllib3.connectionpool:https://rgtdb.com:443 "GET /events/json?search=&offset=0&limit=200 HTTP/1.1" 200 None
DEBUG:cachecontrol.controller:Updating cache with response from "https://rgtdb.com/events/json?search=&offset=0&limit=200"
DEBUG:cachecontrol.controller:Caching b/c of expires header
2021-02-27 14:05:10.825849


In [2]:
response.json().keys()

dict_keys(['total', 'rows'])

## Convert JSON to Pandas Dataframe

Not necessary, but hey, pretending to be a data scientist feels cool.

In [3]:
df = pd.json_normalize(response.json(), 'rows')
df

,name,startAt,detailsUrl,tags,signUps,distance,elevationGain,elevationLost,roadName,roadDetailsUrl,ranked
0,Breakfast Club,02-27 12:00,/events/59721,[groupride],20,30.87 km,76 m,76 m,Borrego Springs,/courses/97,False
1,Saturday Ladies Ride,02-27 12:30,/events/72489,[groupride],2,32.77 km,369 m,114 m,#100 Cheddar,/courses/143710,False
2,Wacky Races 29,02-27 13:00,/events/70573,[groupride],2,40.50 km,1 m,3.25 km,Everest Hairpin Desc,/courses/105756,False
3,Mt Greylock MA,02-27 13:00,/events/70816,[groupride],10,48.21 km,1.19 km,420 m,Virtual Mt Greylock,/courses/140238,False
4,Wacky Races 30,02-27 13:30,/events/69548,[groupride],2,29.35 km,131 m,131 m,Neuron,/courses/118479,False
...,...,...,...,...,...,...,...,...,...,...,...
195,#100 Aberdare,03-16 12:30,/events/62470,[groupride],1,44.32 km,1.15 km,939 m,#100 Aberdare,/courses/111497,False
196,Herd Take it Easy Tuesday,03-16 12:30,/events/69722,[groupride],1,24.41 km,17 m,12 m,Sunk Island To Spurn,/courses/98661,False
197,Tour Series : race 3 (a),03-16 17:30,/events/68042,[race],3,28.28 km,195 m,195 m,Aberystwyth Circuit,/courses/108729,True
198,Tour Series : race 3 (b),03-16 17:30,/events/68044,[race],5,28.28 km,195 m,195 m,Aberystwyth Circuit,/courses/108729,True


## Convert start time into datetime format - and guess at the year

This will be an issue every year around new year.

In [4]:

this_year = str(dt.today().year)
df['date'] = pd.to_datetime(df['startAt'] + ' ' + this_year, format='%m-%d %H:%M %Y', utc=True)
df.set_index('date', inplace=True)


In [5]:
df.head()

,name,startAt,detailsUrl,tags,signUps,distance,elevationGain,elevationLost,roadName,roadDetailsUrl,ranked
date,,,,,,,,,,,
2021-02-27 12:00:00+00:00,Breakfast Club,02-27 12:00,/events/59721,[groupride],20,30.87 km,76 m,76 m,Borrego Springs,/courses/97,False
2021-02-27 12:30:00+00:00,Saturday Ladies Ride,02-27 12:30,/events/72489,[groupride],2,32.77 km,369 m,114 m,#100 Cheddar,/courses/143710,False
2021-02-27 13:00:00+00:00,Wacky Races 29,02-27 13:00,/events/70573,[groupride],2,40.50 km,1 m,3.25 km,Everest Hairpin Desc,/courses/105756,False
2021-02-27 13:00:00+00:00,Mt Greylock MA,02-27 13:00,/events/70816,[groupride],10,48.21 km,1.19 km,420 m,Virtual Mt Greylock,/courses/140238,False
2021-02-27 13:30:00+00:00,Wacky Races 30,02-27 13:30,/events/69548,[groupride],2,29.35 km,131 m,131 m,Neuron,/courses/118479,False


## Use icalendar package to create ICAL format events

* GitHub: https://github.com/collective/icalendar


In [6]:
from datetime import timedelta
from icalendar import vCalAddress, vText
from icalendar import Calendar, Event
import pytz

cal = Calendar()
cal.add('prodid', '-//My calendar product//mxm.com//')
cal.add('version', '2.0')

for index, row in df.iterrows():
    print(index, row['name'], row['tags'])
    event = Event()
    event['uid'] = row['detailsUrl']
    event.add('summary', str(row['name']) + ' ' + str(row['tags']) + ' ' + str(row['signUps']))
    event.add('dtstart', index)
    event.add('dtend', index  + timedelta(hours=1))
    event.add('url', 'https://rgtdb.com' + row['detailsUrl'])
    event.add('description', row['distance'] + ' ' + 'https://rgtdb.com' + row['detailsUrl'])
    event.add('color', 'Tomato')
    event['location'] = vText(row['roadName'])

    cal.add_component(event)

2021-02-27 12:00:00+00:00 Breakfast Club ['groupride']
2021-02-27 12:30:00+00:00 Saturday Ladies Ride ['groupride']
2021-02-27 13:00:00+00:00 Wacky Races 29 ['groupride']
2021-02-27 13:00:00+00:00 Mt Greylock MA ['groupride']
2021-02-27 13:30:00+00:00 Wacky Races 30 ['groupride']
2021-02-27 14:00:00+00:00 Wacky Races 31 ['groupride']
2021-02-27 14:00:00+00:00 J2/9 TT 25Mile ['race', 'itt']
2021-02-27 14:00:00+00:00 NCRA Rockingham Speedway ['race']
2021-02-27 14:00:00+00:00 Weekend Warrior ['groupride']
2021-02-27 14:30:00+00:00 Wacky Races 32 ['groupride']
2021-02-27 15:00:00+00:00 Wacky Races 33 ['groupride']
2021-02-27 15:00:00+00:00 Lou's Saturday Group Ride ['groupride']
2021-02-27 15:00:00+00:00 Chain Gang ['groupride']
2021-02-27 15:00:00+00:00 Napoleon Dolomite ['race']
2021-02-27 15:30:00+00:00 OTR Flemish For Beginners ['race']
2021-02-27 15:30:00+00:00 Wacky Races 34 ['groupride']
2021-02-27 16:00:00+00:00 Wacky Races 35 ['groupride']
2021-02-27 16:00:00+00:00 Vueltina Astur

## Write to File

Can use to manually import into Google, other calendars

In [7]:
import tempfile, os
f = open('./rgt_events.ics', 'wb')
f.write(cal.to_ical())
f.close()

## Use gcsa for Simplified Access to Google Calendar API

* GitHub: https://github.com/kuzmoyev/google-calendar-simple-api
* Docs: https://google-calendar-simple-api.readthedocs.io/en/latest/index.html

Need to add socket timeout of 5 minutes due to slow response on my machine

### Shared Calendar

* Calendar ID: 3e8gau8bommfjk33j92rv5k7q0@group.calendar.google.com
* Google sharing link: https://calendar.google.com/calendar/u/0?cid=M2U4Z2F1OGJvbW1mamszM2o5MnJ2NWs3cTBAZ3JvdXAuY2FsZW5kYXIuZ29vZ2xlLmNvbQ
* ICS format for e.g., Apple Calendar: https://calendar.google.com/calendar/ical/3e8gau8bommfjk33j92rv5k7q0%40group.calendar.google.com/public/basic.ics


In [8]:
from gcsa.event import Event as gcEvent
from gcsa.google_calendar import GoogleCalendar
from gcsa.recurrence import Recurrence, DAILY, SU, SA

import socket
socket.setdefaulttimeout(300) # 5 minutes

EMAIL_FOR_CAL = '3e8gau8bommfjk33j92rv5k7q0@group.calendar.google.com'

calendar = GoogleCalendar(EMAIL_FOR_CAL)


## Cleanup: Delete All Events from Google Calendar

Uncomment to clean up calendar.

In [9]:
#calendar.clear() # This gives an error in the Google API

#for event in calendar.get_events(time_min=df.index.min() - datetime.timedelta(days=1), time_max=df.index.max() + datetime.timedelta(days=1), timezone='UTC'):
#    print('Deleting:', event, event.event_id)
#    calendar.delete_event(event)

## Find existing events, mark for update instead of creation

In [10]:
import re

reExtractName = re.compile(" \[\'.*")

df['cal_id'] = None
df['event_obj'] = None
df['found'] = False

for event in calendar.get_events(time_min=df.index.min() - datetime.timedelta(days=1), time_max=df.index.max() + datetime.timedelta(days=1), timezone='UTC'):

    print(event)

    rideName = reExtractName.sub("", event.summary) # Get substring from event summary with just the name

    df.loc[(df['name'] == rideName) & (df.index == event.start), ['found', 'cal_id', 'event_obj']] = [True, event.event_id, event]


2021-02-28 08:00:00+00:00 - R.Barrett's Race ['race', 'itt'] 4/39.99 km/443 m
2021-03-01 12:30:00+00:00 - #100 Winnats and Peaslows ['groupride'] 1/14.18 km/451 m
2021-03-01 19:30:00+00:00 - #100 Jiggers Bank ['groupride'] 1/24.95 km/515 m
2021-03-02 12:30:00+00:00 - #100 The Burway ['groupride'] 1/22.39 km/602 m
2021-03-02 19:30:00+00:00 - #100 Mow Cop ['groupride'] 1/13.86 km/261 m
2021-03-03 12:30:00+00:00 - #100 Halifax Shibden ['groupride'] 1/15.12 km/520 m
2021-03-03 19:30:00+00:00 - #100 South Yorkshire ['groupride'] 1/35.25 km/1.02 km
2021-03-04 12:30:00+00:00 - #100 Terrace Hill ['groupride'] 1/11.07 km/108 m
2021-03-04 19:30:00+00:00 - #100 Dales Part 1 ['groupride'] 1/33.41 km/910 m
2021-03-05 12:30:00+00:00 - #100 Dales Part 2 ['groupride'] 1/18.20 km/550 m
2021-03-05 18:30:00+00:00 - R2G Friday Night Crit ['race'] 1/23.58 km/238 m
2021-03-05 19:30:00+00:00 - #100 Dales Part 3 ['groupride'] 1/33.34 km/1.15 km
2021-03-06 06:30:00+00:00 - Taith de Cymru Stage1 ITT ['race', 'i

In [11]:
df.loc[df['found'] == True]


,name,startAt,detailsUrl,tags,signUps,distance,elevationGain,elevationLost,roadName,roadDetailsUrl,ranked,cal_id,event_obj,found
date,,,,,,,,,,,,,,
2021-02-27 12:00:00+00:00,Breakfast Club,02-27 12:00,/events/59721,[groupride],20,30.87 km,76 m,76 m,Borrego Springs,/courses/97,False,qvhonq4klttjdnn4617gu5qej8,2021-02-27 12:00:00+00:00 - Breakfast Club ['g...,True
2021-02-27 12:30:00+00:00,Saturday Ladies Ride,02-27 12:30,/events/72489,[groupride],2,32.77 km,369 m,114 m,#100 Cheddar,/courses/143710,False,v9klqclh7f734u4mu0ams0ff4o,2021-02-27 12:30:00+00:00 - Saturday Ladies Ri...,True
2021-02-27 13:00:00+00:00,Wacky Races 29,02-27 13:00,/events/70573,[groupride],2,40.50 km,1 m,3.25 km,Everest Hairpin Desc,/courses/105756,False,vcs1rt0d81bebnriikgd2bn3i0,2021-02-27 13:00:00+00:00 - Wacky Races 29 ['g...,True
2021-02-27 13:00:00+00:00,Mt Greylock MA,02-27 13:00,/events/70816,[groupride],10,48.21 km,1.19 km,420 m,Virtual Mt Greylock,/courses/140238,False,84utkk220v1i661jpqfdum5k9g,2021-02-27 13:00:00+00:00 - Mt Greylock MA ['g...,True
2021-02-27 13:30:00+00:00,Wacky Races 30,02-27 13:30,/events/69548,[groupride],2,29.35 km,131 m,131 m,Neuron,/courses/118479,False,e79o5br095373re2l0ap02o570,2021-02-27 13:30:00+00:00 - Wacky Races 30 ['g...,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-10 19:30:00+00:00,#100 The Cairnwell,03-10 19:30,/events/62398,[groupride],1,14.93 km,417 m,133 m,#100 The Cairnwell,/courses/111553,False,gdnr4gil94aqn2jrt5a80nlv8c,2021-03-10 19:30:00+00:00 - #100 The Cairnwell...,True
2021-03-11 12:30:00+00:00,#100 The Lecht,03-11 12:30,/events/62400,[groupride],1,14.06 km,412 m,176 m,#100 The Lecht,/courses/111549,False,m1on2rpmc8q790cfm232vebk7c,2021-03-11 12:30:00+00:00 - #100 The Lecht ['g...,True
2021-03-11 17:30:00+00:00,Tour Series : race 2 (b),03-11 17:30,/events/68041,[race],7,30.85 km,361 m,361 m,Croydon Tour Series,/courses/127341,True,6v2oof488nkcd66utbrhbtf3n0,2021-03-11 17:30:00+00:00 - Tour Series : race...,True


In [12]:
df.loc[df['found'] == False]

,name,startAt,detailsUrl,tags,signUps,distance,elevationGain,elevationLost,roadName,roadDetailsUrl,ranked,cal_id,event_obj,found
date,,,,,,,,,,,,,,
2021-03-03 05:00:00+00:00,The Recovery Ride,03-03 05:00,/events/71323,[groupride],1,30.87 km,76 m,76 m,Borrego Springs,/courses/97,False,None,None,False
2021-03-10 05:00:00+00:00,The Recovery Ride,03-10 05:00,/events/71324,[groupride],1,30.87 km,76 m,76 m,Borrego Springs,/courses/97,False,None,None,False
2021-03-12 12:30:00+00:00,#100 Rest+Be Thankful,03-12 12:30,/events/62461,[groupride],1,22.04 km,454 m,454 m,#100 Rest&BeThankful,/courses/111541,False,None,None,False
2021-03-12 18:00:00+00:00,8bar crit Berlin - MEN - March,03-12 18:00,/events/40557,[race],8,28.68 km,0 m,0 m,Tempelhof Airport,/courses/105,False,None,None,False
2021-03-12 18:00:00+00:00,SDW Friday Frenzy #5,03-12 18:00,/events/72975,[race],4,20.46 km,250 m,250 m,Fordcombe Loop,/courses/142756,True,None,None,False
2021-03-12 18:30:00+00:00,R2G Friday Night Crit,03-12 18:30,/events/70075,[race],1,20.10 km,177 m,177 m,Forge Valley,/courses/137350,True,None,None,False
2021-03-12 19:00:00+00:00,8bar crit Berlin - WOMEN - March,03-12 19:00,/events/40560,[race],7,19.12 km,0 m,0 m,Tempelhof Airport,/courses/105,False,None,None,False
2021-03-13 06:00:00+00:00,WKG's SDW. Stage 1.,03-13 06:00,/events/60719,[race],4,32.98 km,587 m,504 m,SDW Winchester To QE,/courses/91162,False,None,None,False
2021-03-13 09:00:00+00:00,Virtuslo Medio Fondo,03-13 09:00,/events/49804,[race],5,93.19 km,1.37 km,1.37 km,MF Adria,/courses/94148,False,None,None,False


## Add All Events to Google Calendar

In [13]:
from gcsa.event import Event as gcEvent

for index, row in df.iterrows():

    evntSummary = str(str(row['name']) + ' ' + str(row['tags']) + ' ' + str(row['signUps']) + '/' + row['distance'] + '/' + row['elevationGain'])

    evntDescription = 'Signups: ' +  str(row['signUps']) + '\n' + 'Distance: ' + row['distance'] + '\n' +  'Elevation gain: ' + row['elevationGain'] + '\n' + 'Descent: ' + row['elevationLost'] + '\n' + 'https://rgtdb.com' + str(row['detailsUrl'])

    evntString = row['detailsUrl'].replace('/events/', '')

    if row['found'] == False:

        print('+New event: ', index, row['name'], row['tags'], evntString)

        evntColor = '1'

        if "groupride" in row['tags']:
            evntColor = '2'
        elif 'pro' in row['tags']:
            evntColor = '3'
        elif 'elimination' in row['tags']:
            evntColor = '4'
        elif "itt" in row['tags']:
            evntColor = '5'
        elif "race" in row['tags']:
            evntColor = '6'

        event = gcEvent(
            evntSummary,
            start=index,
            timezone='UTC',
            location=str(row['roadName']),
            description=evntDescription,
            event_id=evntString,
            color = evntColor
        )

        print('ID before add:', event.event_id)
        ret_event = calendar.add_event(event)
        print('ID after add:', event.event_id, 'Returned event ID:', ret_event.event_id)

    else:

        print('-Updating event: ', index, row['name'], row['tags'], eventString)

        event = row['event_obj']

        event.summary = evntSummary
        event.description = evntDescription

        calendar.update_event(event)

NameError: name 'eventString' is not defined

In [709]:
df.loc[df['signUps'] > 9]

,name,startAt,detailsUrl,tags,signUps,distance,elevationGain,elevationLost,roadName,roadDetailsUrl,ranked,cal_id,event_obj,found
date,,,,,,,,,,,,,,
2021-02-27 08:30:00+00:00,Mountains of Tramuntana,02-27 08:30,/events/68182,[race],73,99.77 km,2.21 km,2.04 km,Mountains Of Tramunt,/courses/133754,False,tgshj9717pvrgioroq12do02sk,2021-02-27 08:30:00+00:00 - Mountains of Tramu...,True
2021-02-27 09:00:00+00:00,Virtuslo Medio Fondo,02-27 09:00,/events/49799,[race],11,93.09 km,1.39 km,1.29 km,MF Kras,/courses/94140,True,rs7rf6nb4pu9nr74hgdfekmn5g,2021-02-27 09:00:00+00:00 - Virtuslo Medio Fon...,True
2021-02-27 10:00:00+00:00,CTT 25 mile TT,02-27 10:00,/events/71544,"[race, itt]",16,40.43 km,137 m,274 m,CTT R25/3H,/courses/107825,False,ua9buo0jcsmpnrhmeapocuqd4g,2021-02-27 10:00:00+00:00 - CTT 25 mile TT ['r...,True
2021-02-27 14:00:00+00:00,J2/9 TT 25Mile,02-27 14:00,/events/71119,"[race, itt]",33,40.31 km,161 m,191 m,J2_9 TT,/courses/140278,False,5hk4v7dd6dmn2o197sh87dc4fk,2021-02-27 14:00:00+00:00 - J2/9 TT 25Mile ['r...,True
2021-02-27 14:00:00+00:00,NCRA Rockingham Speedway,02-27 14:00,/events/71619,[race],13,41.04 km,208 m,208 m,NCRA Rocko Speedway,/courses/128635,False,jco2f7a6j9j0p4ftgnnmkjt250,2021-02-27 14:00:00+00:00 - NCRA Rockingham Sp...,True
2021-02-27 15:00:00+00:00,Lou's Saturday Group Ride,02-27 15:00,/events/71288,[groupride],11,30.87 km,76 m,76 m,Borrego Springs,/courses/97,False,b6fv6iedc9tgiog3t4s9v1l7v4,2021-02-27 15:00:00+00:00 - Lou's Saturday Gro...,True
2021-02-27 15:30:00+00:00,OTR Flemish For Beginners,02-27 15:30,/events/70276,[race],14,37.65 km,88 m,87 m,OTR Krombeke,/courses/56431,True,26bpa9lvf2s4169bbjakq21v4k,2021-02-27 15:30:00+00:00 - OTR Flemish For Be...,True
2021-02-27 16:00:00+00:00,Vueltina Asturias Stage3,02-27 16:00,/events/67750,[race],23,23.12 km,436 m,292 m,Ribadesella-Alto La,/courses/117041,False,0dldbsghq3ou5rndbh4fjhn8lc,2021-02-27 16:00:00+00:00 - Vueltina Asturias ...,True
2021-02-28 09:00:00+00:00,GFNS JENSIE E-FONDO,02-28 09:00,/events/56538,[race],60,43.43 km,1.30 km,616 m,7. E Fondo Jensi V3 Test,/courses/107793,False,eg58k2ml3tm1n9cb9ecu7dcutg,2021-02-28 09:00:00+00:00 - GFNS JENSIE E-FOND...,True


In [710]:
df.loc[df['name'].str.startswith('Wacky Races')]

,name,startAt,detailsUrl,tags,signUps,distance,elevationGain,elevationLost,roadName,roadDetailsUrl,ranked,cal_id,event_obj,found
date,,,,,,,,,,,,,,
2021-02-27 06:00:00+00:00,Wacky Races 15,02-27 06:00,/events/68875,[groupride],1,31.29 km,489 m,492 m,French Curves,/courses/119225,False,d0ssqd7jm8dcapo0cjcb1ltu3s,2021-02-27 06:00:00+00:00 - Wacky Races 15 ['g...,True
2021-02-27 06:30:00+00:00,Wacky Races 16,02-27 06:30,/events/68876,[groupride],1,34.88 km,498 m,498 m,Star3,/courses/120097,False,ldsc9cu262frjun8ufacghj0ds,2021-02-27 06:30:00+00:00 - Wacky Races 16 ['g...,True
2021-02-27 07:00:00+00:00,Wacky Races 17,02-27 07:00,/events/68877,[groupride],1,18.91 km,257 m,257 m,Alcatraz_Loop,/courses/134262,False,vi79fle65kafjp69o712enu2lo,2021-02-27 07:00:00+00:00 - Wacky Races 17 ['g...,True
2021-02-27 07:30:00+00:00,Wacky Races 18,02-27 07:30,/events/68881,[groupride],1,39.94 km,543 m,543 m,Mitochondrion,/courses/117505,False,3j5bh920l83gpnp35r4gknd82c,2021-02-27 07:30:00+00:00 - Wacky Races 18 ['g...,True
2021-02-27 08:00:00+00:00,Wacky Races 19,02-27 08:00,/events/68885,[groupride],2,34.22 km,589 m,589 m,Spiro5.1,/courses/132221,False,blgp940vmvk2a9165r3cutfiv0,2021-02-27 08:00:00+00:00 - Wacky Races 19 ['g...,True
2021-02-27 08:30:00+00:00,Wacky Races 20,02-27 08:30,/events/68886,[groupride],1,25.45 km,672 m,674 m,SMR Double Helix,/courses/111901,False,i5stp307k8bm53stbn12ggvt9k,2021-02-27 08:30:00+00:00 - Wacky Races 20 ['g...,True
2021-02-27 09:00:00+00:00,Wacky Races 21,02-27 09:00,/events/68887,[groupride],1,23.92 km,656 m,663 m,Loops And Bends,/courses/125675,False,72r3hmhq8nr6jn2opg824t5q08,2021-02-27 09:00:00+00:00 - Wacky Races 21 ['g...,True
2021-02-27 09:30:00+00:00,Wacky Races 22,02-27 09:30,/events/68888,[groupride],1,32.10 km,706 m,706 m,Crazy Caldera,/courses/134478,False,ac3u221g6dtgv4h024u6cn5s2s,2021-02-27 09:30:00+00:00 - Wacky Races 22 ['g...,True
2021-02-27 10:00:00+00:00,Wacky Races 23,02-27 10:00,/events/68889,[groupride],2,34.07 km,737 m,737 m,Spirograph1.2,/courses/131403,False,g49b8atoh5u7vlo0aj4rnd2vi4,2021-02-27 10:00:00+00:00 - Wacky Races 23 ['g...,True
